In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [4]:
base = pd.read_csv('../data/bank-full.csv', sep=';')
pd.set_option('display.max_columns', None)
base

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown,yes
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown,yes
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,yes
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown,no


In [5]:
y = base.iloc[:,16]
base = base.drop(['y'], axis=1)

In [6]:
base.select_dtypes(include='object')

,job,marital,education,default,housing,loan,contact,month,poutcome
0,management,married,tertiary,no,yes,no,unknown,may,unknown
1,technician,single,secondary,no,yes,no,unknown,may,unknown
2,entrepreneur,married,secondary,no,yes,yes,unknown,may,unknown
3,blue-collar,married,unknown,no,yes,no,unknown,may,unknown
4,unknown,single,unknown,no,no,no,unknown,may,unknown
...,...,...,...,...,...,...,...,...,...
45206,technician,married,tertiary,no,no,no,cellular,nov,unknown
45207,retired,divorced,primary,no,no,no,cellular,nov,unknown
45208,retired,married,secondary,no,no,no,cellular,nov,success
45209,blue-collar,married,secondary,no,no,no,telephone,nov,unknown


In [7]:
base.select_dtypes(exclude='object')

,age,balance,day,duration,campaign,pdays,previous
0,58,2143,5,261,1,-1,0
1,44,29,5,151,1,-1,0
2,33,2,5,76,1,-1,0
3,47,1506,5,92,1,-1,0
4,33,1,5,198,1,-1,0
...,...,...,...,...,...,...,...
45206,51,825,17,977,3,-1,0
45207,71,1729,17,456,2,-1,0
45208,72,5715,17,1127,5,184,3
45209,57,668,17,508,4,-1,0


In [8]:
labelEnc = LabelEncoder()
oneHotEnc = OneHotEncoder()

In [9]:
# Transformando as variáveis categóricas ordinais com Label Encoder.
# As variáveis education e month precisaram ser convertidas manualmente 
# devido a ordem alfabética do nível de educação e dos meses do ano,
# respectivamente.
cat_ord = ['education', 'default', 'housing', 'loan', 'month']

for i in range(len(cat_ord)):
    for j in range(len(base.columns)):
        if cat_ord[i] == base.columns[j]:
            x = cat_ord[i]
            if x == 'education':
                base[x].replace({'unknown':0,'primary':1,'secondary':2,'tertiary':3}, inplace=True)
            elif x == 'month':
                base[x].replace({'jan':1, 'feb':2, 'mar':3, 'apr':4, 'may':5, 'jun':6, 
                        'jul':7, 'aug':8, 'sep':9, 'oct':10, 'nov':11, 'dec':12}, 
                        inplace=True)
            else: 
                base[x] = labelEnc.fit_transform(base[x])

In [10]:
# Transformando as variáveis categóricas nominais com One Hot Encoder, nomeando as
# colunas transformadas, concatenando essas culunas à base de dados e realizando o
# drop das colunas originais.
cat_nom = ['job','marital','contact','poutcome']

for i in range(len(cat_nom)):
    for j in range(len(base.columns)):
        if cat_nom[i] == base.columns[j]:
            x = cat_nom[i]
            feature_array = pd.DataFrame(oneHotEnc.fit_transform(base[[x]]).toarray(), columns=np.array(oneHotEnc.categories_).ravel())
            base = pd.concat([base, feature_array], axis=1)
            base = base.drop([x], axis=1)

In [11]:
base

,age,education,default,balance,housing,loan,day,month,duration,campaign,pdays,previous,admin.,blue-collar,entrepreneur,housemaid,management,retired,self-employed,services,student,technician,unemployed,unknown,divorced,married,single,cellular,telephone,unknown,failure,other,success,unknown
0,58,3,0,2143,1,0,5,5,261,1,-1,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,44,2,0,29,1,0,5,5,151,1,-1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,33,2,0,2,1,1,5,5,76,1,-1,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,47,0,0,1506,1,0,5,5,92,1,-1,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,33,0,0,1,0,0,5,5,198,1,-1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,3,0,825,0,0,17,11,977,3,-1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
45207,71,1,0,1729,0,0,17,11,456,2,-1,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
45208,72,2,0,5715,0,0,17,11,1127,5,184,3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
45209,57,2,0,668,0,0,17,11,508,4,-1,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [12]:
# Função para verificar possível problema de multicolinearidade
def calculate_vif(dataset):
    vif = pd.DataFrame()
    vif['features'] = dataset.columns
    vif['VIF_value'] = [variance_inflation_factor(dataset.values, i) for i in range(dataset.shape[1])]
    return vif

In [13]:
job = base.iloc[:,12:24]
marital = base.iloc[:,24:27]
contact = base.iloc[:,27:30]
poutcome = base.iloc[:,30:35]

In [14]:
print(calculate_vif(job),'\n')
print(calculate_vif(marital),'\n')
print(calculate_vif(contact),'\n')
print(calculate_vif(poutcome))

         features  VIF_value
0          admin.        1.0
1     blue-collar        1.0
2    entrepreneur        1.0
3       housemaid        1.0
4      management        1.0
5         retired        1.0
6   self-employed        1.0
7        services        1.0
8         student        1.0
9      technician        1.0
10     unemployed        1.0
11        unknown        1.0 

   features  VIF_value
0  divorced        1.0
1   married        1.0
2    single        1.0 

    features  VIF_value
0   cellular        1.0
1  telephone        1.0
2    unknown        1.0 

  features  VIF_value
0  failure        1.0
1    other        1.0
2  success        1.0
3  unknown        1.0


In [15]:
# Devolvendo a classe y para a base de dados.
base = pd.concat([base, y], axis=1)

In [16]:
base.head()

,age,education,default,balance,housing,loan,day,month,duration,campaign,pdays,previous,admin.,blue-collar,entrepreneur,housemaid,management,retired,self-employed,services,student,technician,unemployed,unknown,divorced,married,single,cellular,telephone,unknown,failure,other,success,unknown,y
0,58,3,0,2143,1,0,5,5,261,1,-1,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,no
1,44,2,0,29,1,0,5,5,151,1,-1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,no
2,33,2,0,2,1,1,5,5,76,1,-1,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,no
3,47,0,0,1506,1,0,5,5,92,1,-1,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,no
4,33,0,0,1,0,0,5,5,198,1,-1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,no


In [17]:
base['unknown']

,unknown,unknown,unknown
0,0.0,1.0,1.0
1,0.0,1.0,1.0
2,0.0,1.0,1.0
3,0.0,1.0,1.0
4,1.0,1.0,1.0
...,...,...,...
45206,0.0,0.0,1.0
45207,0.0,0.0,1.0
45208,0.0,0.0,0.0
45209,0.0,0.0,1.0


In [18]:
base.drop(['unknown'], axis=1, inplace=True)

In [19]:
base

,age,education,default,balance,housing,loan,day,month,duration,campaign,pdays,previous,admin.,blue-collar,entrepreneur,housemaid,management,retired,self-employed,services,student,technician,unemployed,divorced,married,single,cellular,telephone,failure,other,success,y
0,58,3,0,2143,1,0,5,5,261,1,-1,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,no
1,44,2,0,29,1,0,5,5,151,1,-1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,no
2,33,2,0,2,1,1,5,5,76,1,-1,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,no
3,47,0,0,1506,1,0,5,5,92,1,-1,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,no
4,33,0,0,1,0,0,5,5,198,1,-1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,3,0,825,0,0,17,11,977,3,-1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,yes
45207,71,1,0,1729,0,0,17,11,456,2,-1,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,yes
45208,72,2,0,5715,0,0,17,11,1127,5,184,3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,yes
45209,57,2,0,668,0,0,17,11,508,4,-1,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,no


In [21]:
#base.to_csv('../data/bank-full.csv_v2', index=False)